In [12]:
import pandas as pd

from surprise import reader


In [13]:
df = pd.read_csv("Datasets/data_train.csv")

In [14]:
df[["user", "item"]] = df.Id.str.split("_", expand=True)

In [15]:
df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")

In [18]:
from surprise import Reader

reader = Reader(rating_scale=(1,5))

In [26]:
from surprise import Dataset

data = Dataset.load_from_df(df[["user","item","Prediction"]], reader)

In [27]:
data

In [30]:
from surprise import SVD
from surprise import model_selection
algo = SVD()

In [33]:
model_selection.cross_validate(algo, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0261  1.0264  1.0282  1.0259  1.0303  1.0274  0.0017  
MAE (testset)     0.8182  0.8196  0.8207  0.8181  0.8215  0.8196  0.0013  
Fit time          70.74   68.84   70.45   69.90   65.97   69.18   1.73    
Test time         8.02    3.54    3.67    3.53    3.46    4.44    1.79    


{'test_rmse': array([1.02613722, 1.02636535, 1.02823834, 1.02590192, 1.03029469]),
 'test_mae': array([0.81817137, 0.81963593, 0.82071993, 0.81807393, 0.82146268]),
 'fit_time': (70.73839735984802,
  68.84112548828125,
  70.44979453086853,
  69.90368556976318,
  65.96970796585083),
 'test_time': (8.020948648452759,
  3.540347099304199,
  3.668789863586426,
  3.530874729156494,
  3.4624173641204834)}

In [36]:
df2 = pd.read_csv("Datasets/sample_submission.csv")
df2[["user", "item"]] = df2.Id.str.split("_", expand=True)
df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")
df2

,Id,Prediction,user,item
0,r37_c1,3,37,1
1,r73_c1,3,73,1
2,r156_c1,3,156,1
3,r160_c1,3,160,1
4,r248_c1,3,248,1
...,...,...,...,...
1176947,r9974_c1000,3,9974,1000
1176948,r9977_c1000,3,9977,1000
1176949,r9978_c1000,3,9978,1000
1176950,r9982_c1000,3,9982,1000


In [68]:
pred = algo.predict(37,1, verbose=True)

user: 37         item: 1          r_ui = None   est = 3.86   {'was_impossible': False}
